# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv into dataframe
vacation_csv = "0. city_data.csv"
vacation_df = pd.read_csv(vacation_csv)
vacation_df = vacation_df.drop(columns='Unnamed: 0')
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,Airai,-8.9266,125.4092,72.41,39,48,1.97,TL,1619490043
1,Ucluelet,48.9329,-125.5528,54.00,71,40,6.91,CA,1619490044
2,Qaqortoq,60.7167,-46.0333,35.60,93,75,5.75,GL,1619489961
3,Rikitea,-23.1203,-134.9692,78.13,72,100,6.98,PF,1619490044
4,Puerto Ayora,-0.7393,-90.3518,75.00,90,37,4.97,EC,1619490018


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#variables for heatmap
location = vacation_df[["Lat", "Lng"]]
humidity = vacation_df['Humidity'].astype(int)

# Heatmap
map1 = gmaps.figure()
map1
heat_layer = gmaps.heatmap_layer(location, weights=humidity,dissipating=False, max_intensity=10, point_radius=1)                          
map1.add_layer(heat_layer)
map1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

idealvacation_df = vacation_df.copy()
idealvacation_df = idealvacation_df[idealvacation_df['Max Temp'] >= 70]
idealvacation_df = idealvacation_df[idealvacation_df['Max Temp'] <= 80]  
idealvacation_df = idealvacation_df[idealvacation_df['WindSpeed'] <= 10]
idealvacation_df = idealvacation_df[idealvacation_df['Cloudiness'] == 0]
idealvacation_df = idealvacation_df.dropna()
idealvacation_df = idealvacation_df.reset_index()
idealvacation_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,26,Carnarvon,-24.8667,113.6333,78.80,47,0,3.44,AU,1619489895
1,29,Alta Floresta,-9.8756,-56.0861,73.40,100,0,2.30,BR,1619490051
2,132,Buraidah,26.3260,43.9750,75.20,53,0,6.91,SA,1619490082
3,165,Panzhihua,26.5851,101.7128,77.41,39,0,2.28,CN,1619490092
4,197,Lençóis Paulista,-22.5986,-48.8003,73.40,68,0,3.44,BR,1619490102
5,215,Umluj,25.0213,37.2685,76.08,69,0,7.87,SA,1619489836
6,292,Castanhal,-1.2939,-47.9264,78.80,83,0,2.30,BR,1619490134


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create hotel_df with hotel name column
hotel_df = idealvacation_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
0,Carnarvon,-24.8667,113.6333,78.80,47,0,3.44,AU,1619489895,NaN
1,Alta Floresta,-9.8756,-56.0861,73.40,100,0,2.30,BR,1619490051,NaN
2,Buraidah,26.3260,43.9750,75.20,53,0,6.91,SA,1619490082,NaN
3,Panzhihua,26.5851,101.7128,77.41,39,0,2.28,CN,1619490092,NaN
4,Lençóis Paulista,-22.5986,-48.8003,73.40,68,0,3.44,BR,1619490102,NaN
5,Umluj,25.0213,37.2685,76.08,69,0,7.87,SA,1619489836,NaN
6,Castanhal,-1.2939,-47.9264,78.80,83,0,2.30,BR,1619490134,NaN


In [6]:
#hotels within 5000 meters
hotelsnames = []
cont = 0
# loop through city Latitude and Longitude
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][cont])+' , '+str(hotel_df['Lng'][cont])
    cont = cont + 1
    target_radius = 5000
    target_type = 'lodging'
    params = {"location": target_coordinates,"radius": target_radius,"type": target_type,"key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        response = requests.get(base_url, params=params)
        hotelname_results = response.json()
        hotelsnames.append(hotelname_results['results'][0]['name'])
    except IndexError:
        hotelsnamex.append(np.nan)

#dataframe with hotel names
hotel_df['Hotel Name'] = hotelsnames
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
0,Carnarvon,-24.8667,113.6333,78.80,47,0,3.44,AU,1619489895,Hospitality Carnarvon
1,Alta Floresta,-9.8756,-56.0861,73.40,100,0,2.30,BR,1619490051,Floresta Amazonica Hotel
2,Buraidah,26.3260,43.9750,75.20,53,0,6.91,SA,1619490082,فندق راحة السلام
3,Panzhihua,26.5851,101.7128,77.41,39,0,2.28,CN,1619490092,Electric Power Hotel
4,Lençóis Paulista,-22.5986,-48.8003,73.40,68,0,3.44,BR,1619490102,Saint James Palace Hotel
5,Umluj,25.0213,37.2685,76.08,69,0,7.87,SA,1619489836,Moon light Furnished Units
6,Castanhal,-1.2939,-47.9264,78.80,83,0,2.30,BR,1619490134,ESTAÇÃO HOTEL


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
# Display map
hotelmap = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
hotelmap.add_layer(markers)
hotelmap

Figure(layout=FigureLayout(height='420px'))